In [1]:
from langsmith import Client
from langsmith.evaluation import evaluate
from langsmith.schemas import Example, Run

# Initialize LangSmith client
client = Client()

In [ ]:
dataset = client.create_dataset(
    "Customer Support RAG Dataset",
    description="A dataset for evaluating RAG performance in customer support scenarios"
)

In [ ]:
client.create_examples(
    inputs=[
        {"input": "¿Cuáles son los canales de atención al cliente que ofrece TechnoVerde?"},
        {"input": "¿Cuál es el tiempo de respuesta objetivo para los correos electrónicos?"},
        {"input": "¿Cuánto tiempo tengo para devolver un producto?"},
        {"input": "¿Qué cubre la garantía de los productos TechnoVerde?"},
        {"input": "¿Cómo funciona el programa de lealtad de TechnoVerde?"},
        {"input": "¿Qué medidas toma TechnoVerde para proteger los datos de los clientes?"},
        {"input": "¿Qué tipo de formación reciben los empleados de atención al cliente?"},
        {"input": "¿Cómo maneja TechnoVerde las situaciones de crisis?"},
        {"input": "¿Qué tecnologías innovadoras utiliza TechnoVerde en su atención al cliente?"},
        {"input": "¿Cómo asegura TechnoVerde la accesibilidad en su atención al cliente?"}
    ],
    outputs=[
        {"response": "TechnoVerde S.A. ofrece múltiples canales de atención al cliente: una línea telefónica gratuita (0800-TECNO-VERDE) disponible 24/7, correo electrónico (soporte@technoverde.com), chat en vivo en su sitio web (www.technoverde.com), redes sociales (@TechnoVerdeOficial en Facebook, Twitter e Instagram), y atención presencial en sus tiendas de lunes a sábado de 9:00 a 20:00."},
        {"response": "Según la política de atención al cliente de TechnoVerde, el tiempo de respuesta objetivo para los correos electrónicos es de menos de 4 horas hábiles."},
        {"response": "De acuerdo con la política de reembolsos y devoluciones de TechnoVerde, los clientes pueden devolver productos no utilizados dentro de los 30 días posteriores a la compra para obtener un reembolso completo o un cambio."},
        {"response": "Todos los productos TechnoVerde están cubiertos por una garantía mínima de 2 años. Algunos productos premium tienen garantías extendidas de hasta 5 años. Además, la empresa ofrece soporte técnico gratuito durante toda la vida útil del producto."},
        {"response": "El programa de lealtad de TechnoVerde funciona de la siguiente manera: los clientes son automáticamente inscritos al realizar su primera compra, ganan 1 punto por cada $10 gastados, y hay cuatro niveles de membresía (Verde, Plata, Oro y Platino) con beneficios crecientes. Los beneficios incluyen acceso anticipado a nuevos productos, invitaciones a eventos exclusivos y descuentos especiales."},
        {"response": "TechnoVerde toma varias medidas para proteger los datos de los clientes. Solo recopilan los datos necesarios para proporcionar sus servicios y mejorar la experiencia del cliente. Todos los datos se almacenan de forma segura en servidores encriptados y se implementan medidas de seguridad de vanguardia para proteger la información contra accesos no autorizados."},
        {"response": "Los empleados de atención al cliente de TechnoVerde reciben un programa de formación inicial intensivo de 4 semanas que cubre conocimiento de productos y servicios, habilidades de comunicación y resolución de problemas, políticas y procedimientos de la empresa, y uso de sistemas de atención al cliente. Además, se ofrecen programas de formación continua mensuales para mantenerlos actualizados."},
        {"response": "TechnoVerde mantiene un plan detallado de continuidad del negocio para garantizar la atención al cliente en casos de crisis como desastres naturales, fallos tecnológicos o pandemias. En situaciones de crisis, se comprometen a informar proactivamente a los clientes sobre cualquier interrupción del servicio, proporcionar actualizaciones regulares y ofrecer soluciones alternativas cuando sea posible."},
        {"response": "TechnoVerde utiliza varias tecnologías innovadoras en su atención al cliente, incluyendo soluciones de Inteligencia Artificial y chatbots para proporcionar respuestas instantáneas y analizar el sentimiento del cliente. También implementan tecnología de Realidad Aumentada (RA) para guiar a los clientes en la instalación y resolución de problemas de productos, así como para proporcionar demostraciones virtuales."},
        {"response": "TechnoVerde asegura la accesibilidad en su atención al cliente mediante un enfoque de diseño universal. Esto incluye un sitio web y aplicaciones móviles compatibles con lectores de pantalla, opciones de comunicación para clientes con discapacidad auditiva o del habla, y materiales de soporte en múltiples idiomas y formatos. Además, todo el personal de atención al cliente recibe formación regular sobre diversidad e inclusión."}
    ],
    dataset_id=dataset.id
)


In [2]:
#AI App

from agent import Agent
from langchain_chroma import Chroma
from tools import create_retriever_tool_from_vectorstore
from langchain_core.messages import HumanMessage
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import AIMessage
persist_directory = "./chroma_db"
vectorstore = Chroma(
    collection_name="rag-chroma",
    embedding_function=OpenAIEmbeddings(),
    persist_directory=persist_directory
)

tools = [create_retriever_tool_from_vectorstore(vectorstore)]

In [8]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate([
        ("system", "Sos un asistente que responde preguntas sobre la empresa TechnoVerde S.A. Para preguntas relacionadas a la empresa, responde utilizando la informacion que tenes disponible sobre la misma, no inventes informacion. Si no conoces la respuesta, simplemente decí que no lo sabes y disculpate por no poder ayudar"),
    ])

In [19]:
def start_conversation(input_dict):
    try:
        agent = Agent(model_type="groq", prompt=template, tools=tools)
        human_message = HumanMessage(content=input_dict['input'])
        state = agent.invoke([human_message])
        
        if isinstance(state, dict) and "messages" in state and len(state["messages"]) > 0:
            last_message = state["messages"][-1]
            if isinstance(last_message, AIMessage):
                return {"answer": last_message.content}
        
        return {"answer": "No AI response found in the state."}
    except Exception as e:
        return {"error": str(e)}

In [17]:
start_conversation({"input": "Para que empresa trabajas?"})


{'answer': 'Trabajo para TechnoVerde S.A. ¿En qué puedo ayudarte hoy?'}

In [11]:
start_conversation({"input": "Hola, como estas?"})

{'answer': 'No AI response found in the state.'}

In [20]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

answer_correctness_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "answer_correctness": """Is the Assistant's Answer aligned with the Ground Truth Answer? A score of [[1]] means that the
            Assistant answer contains is not at all based upon / grounded in the Groun Truth Answer. A score of [[5]] means 
            that the Assistant answer contains some information (e.g., a hallucination) that is not captured in the Ground Truth 
            Answer. A score of [[10]] means that the Assistant answer is fully based upon the in the Ground Truth Answer."""
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
        "key": "answer_correctness"
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["response"],
        "input": example.inputs["input"],
    },
)

dataset_name = "Customer Support RAG Dataset"
experiment_results = evaluate(
    start_conversation,
    data=dataset_name,
    evaluators=[answer_correctness_evaluator],
    experiment_prefix="rag-answer-correctness-llama3.1",

)


View the evaluation results for experiment: 'rag-answer-correctness-llama3.1-1fadfe67' at:
https://smith.langchain.com/o/0b56ec53-f91c-40f8-a254-2f8b48ff6eb5/datasets/dddbf6f3-7f3c-4b14-81e2-d6e19d876817/compare?selectedSessions=e9cbd436-e840-4455-8397-9baf6133d811




0it [00:00, ?it/s]Parent run e8fb09db-7cac-43df-9c49-5e3e3fc34918 not found for run 297e8dd0-b59e-446f-94d2-2e3fe400f373. Treating as a root run.
Parent run 28d71e14-f776-400b-93c6-5d9960216689 not found for run 3943b5db-d675-49c7-8726-d6aa1db45673. Treating as a root run.
Parent run 92d5a4f3-a228-4070-840e-b7d44986d6c1 not found for run ef0bd5b4-6e8c-4a99-ad3b-f16ec2975307. Treating as a root run.
Parent run ed8960a6-1829-4fff-aef6-a60d4738f677 not found for run ffa12665-c2e0-4954-8281-6204ec3738fe. Treating as a root run.
Parent run ada18d8b-5b35-4141-ad41-83a303917788 not found for run aa921604-d75e-450d-a132-0879794240bf. Treating as a root run.
Parent run 9f73bcf6-4965-41b5-95b5-283526f5637f not found for run 0b6e8321-dc8e-4538-824c-2ed5348ce353. Treating as a root run.
Parent run bb18842d-72fb-4e9f-b7da-ade1b3019740 not found for run eaa814ee-aafa-4b17-8256-f8354b4c9c52. Treating as a root run.
Parent run 00263c56-996c-4061-afcf-af4b5a73a0ee not found for run 45ddbbe1-6785-41c8-a